In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# InteractiveShell.ast_node_interactivity = "last_expr"

In [80]:
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import os
import folium
from folium.plugins import HeatMap

print(folium.__version__)

0.9.1


In [66]:
# import data
counties_geocodes_df = pd.read_csv('../Data/counties_geocodes.csv').drop(columns=['Unnamed: 0', 
                                                                                  'index','state_code',
                                                                                  'county_code'])
counties_summary_2019 = pd.read_csv('../Data/counties_summary.csv')

In [67]:
counties_summary_2019.county.unique()

array(['Afrique', 'Akjoujt', 'Aleg', 'Amourj', 'Aoujeft', 'Arafat',
       'Asie', 'Atar', 'Aïoun', 'Bababé', 'Barkéol', 'Bassiknou',
       'Bennechab', 'Bir Moghrein', 'Boghé', 'Boumdeid', 'Boutilimit',
       'Chami', 'Chinguitti', 'Dar Naïm', 'Djiguenni', 'El Mina',
       'Europe', "F'Deirick", 'Ghabou', 'Guerou', 'Kankoussa', 'Kaédi',
       'Keur Macène', 'Kiffa', 'Koubenni', 'Ksar', "M'Bagne", "M'Bout",
       'Maghama', 'Magtalahjar', 'Mederdra', 'Monguel', 'Moudjeria',
       "N'beiket Lahwach", 'Nouadhibou', 'Néma', 'Ouad Naga', 'Ouadane',
       'Oualata', 'Ould Yengé', "R'Kiz", 'Riyad', 'Rosso', 'Sebkha',
       'Sélibaby', 'Tamcheket', 'Tevragh Zeina', 'Teyaret', 'Tichit',
       'Tidjikja', 'Timbedra', 'Tintane', 'Toujounine', 'Zoueratt'],
      dtype=object)

In [68]:
# drop overseas locations 
to_drop = ['Afrique', 'Asie', 'Europe']
mr_counties_summary = counties_summary_2019[~counties_summary_2019['county'].isin(to_drop)]
mr_counties_summary.head()

,county,total_registered,voted,void,neutral,suffrage,perct_turnout,biram,kane,ghazouani,wavi,mewloud,boubacar
1,Akjoujt,9661.0,5933.0,95.0,64.0,5774.0,0.614119,300,288,3422,14,128,1622
2,Aleg,52156.0,32328.0,935.0,137.0,31256.0,0.619833,2324,286,20143,49,427,8027
3,Amourj,35019.0,20004.0,670.0,68.0,19266.0,0.571233,712,84,16804,56,44,1566
4,Aoujeft,9479.0,5786.0,70.0,24.0,5692.0,0.610402,75,21,4395,9,396,796
5,Arafat,64009.0,46354.0,1289.0,508.0,44557.0,0.724179,7486,2859,19730,318,1725,12439


In [69]:
counties_geocodes_df.head()

,county,lat,lng
0,Néma,16.614370,-7.259891
1,Amourj,15.848233,-7.265286
2,Bassiknou,15.861876,-5.954182
3,Djiguenni,15.893680,-8.572102
4,Timbedra,16.241659,-8.172099


In [120]:
# concat two dataframes 
mr_counties_data = pd.merge(counties_geocodes_df, mr_counties_summary, on='county', how='outer')
mr_counties_data['total_registered'] = mr_counties_data['total_registered'].apply(lambda x: x*100)

In [121]:
def generateBaseMap(default_location=[18.440050, -9.495773], default_zoom_start=6):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [125]:
base_map = generateBaseMap()
HeatMap(data=mr_counties_data[['lat', 'lng', 'total_registered']].groupby(['lat', 'lng']).sum().reset_index().values.tolist(), radius=15, max_zoom=30).add_to(base_map)
base_map